In [0]:
# sprint19_Unet セグメンテーション　boostnoteに整理した　

In [0]:
"""
全体の流れ
1.実行環境を整える。（マウント、kaggleコマンドのインストール）
2.使用する塩データの取得（アンジップ）
3.Unetのモデルをクローンする（アンジップデータを塩データをunet配下へ移動し学習出来るようにしておく）
4.main.pyを動かす（それに伴って、data.pyやファイル名を変更する）
"""

In [0]:
# kaggleの塩画像データを使いたいのでkaggleコマンドでダウンロード
!pip install kaggle

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
mkdir .kaggle

In [0]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"shuji0001","key":"bfae10a59d1e896cb7d279ddd1274e89"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

# 新しいセクション

In [0]:
# 2回目の実施はここから。
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
#  ここでコケるので上のセルでもう一度流し直す。コケる原因は不明。
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
# simpsonデータセットのAPIcommandを叩いてデータセットをダウンロードする
# APIcommandはこちらのサイトに。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

# 塩データをDLしたいのでsimpson データを引っ張ってくる部分を変更する。

! kaggle competitions download -c tgs-salt-identification-challenge

  0% 0.00/922k [00:00<?, ?B/s]
100% 922k/922k [00:00<00:00, 60.9MB/s]
  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 84.6MB/s]
 86% 187M/217M [00:01<00:00, 95.3MB/s]
100% 217M/217M [00:01<00:00, 132MB/s] 
  0% 0.00/322k [00:00<?, ?B/s]
100% 322k/322k [00:00<00:00, 103MB/s]
 80% 33.0M/41.4M [00:00<00:00, 36.2MB/s]
100% 41.4M/41.4M [00:00<00:00, 118MB/s] 
 71% 27.0M/37.9M [00:00<00:00, 25.3MB/s]
100% 37.9M/37.9M [00:00<00:00, 85.4MB/s]
 89% 145M/163M [00:01<00:00, 69.2MB/s]
100% 163M/163M [00:01<00:00, 106MB/s] 


In [0]:
# train用の塩画像データセットを解凍
# 実行するとimages と makes というディレクトリが作成されそこに保存される。
# 使うのはimagesに入ってる方のデータ
!unzip /content/train.zip -d train_solt

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: train_solt/images/b012e9ebb0.png  
  inflating: train_solt/images/d3034e7a60.png  
  inflating: train_solt/images/ae2ef70d51.png  
  inflating: train_solt/images/1af44a9db0.png  
  inflating: train_solt/images/41cfd4b320.png  
  inflating: train_solt/images/853e89d0d3.png  
  inflating: train_solt/images/2a484baa26.png  
  inflating: train_solt/images/92e617896e.png  
  inflating: train_solt/images/b70f0ab0c2.png  
  inflating: train_solt/images/e8757626a1.png  
  inflating: train_solt/images/6a3d92ce59.png  
  inflating: train_solt/images/127ef4105f.png  
  inflating: train_solt/images/d21012f5c1.png  
  inflating: train_solt/images/1cb9549d2c.png  
  inflating: train_solt/images/e030bdae65.png  
  inflating: train_solt/images/0cf7155eb6.png  
  inflating: train_solt/images/badb3ea766.png  
  inflating: train_solt/images/9bd58061d6.png  
  inflating: train_solt/images/2b1e71c2a3.png  
  inflating: train_solt/images/28d42de29f.png  
  inflat

In [0]:
# train用の塩画像データセットを解凍
# 実行するとimages と makes というディレクトリが作成されそこに保存される。
# 使うのはimagesに入ってる方のデータ
!unzip /content/test.zip -d test_solt

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: test_solt/images/187ecc32d5.png  
  inflating: test_solt/images/54538419f5.png  
  inflating: test_solt/images/93614d4b95.png  
  inflating: test_solt/images/50a91adce8.png  
  inflating: test_solt/images/97d23cea10.png  
  inflating: test_solt/images/ca04c19c0b.png  
  inflating: test_solt/images/d168218bac.png  
  inflating: test_solt/images/a670e833a4.png  
  inflating: test_solt/images/a964a0f499.png  
  inflating: test_solt/images/371bf59030.png  
  inflating: test_solt/images/fc74f77bae.png  
  inflating: test_solt/images/f63dc1e3be.png  
  inflating: test_solt/images/b807539a5f.png  
  inflating: test_solt/images/1dfcbf4bca.png  
  inflating: test_solt/images/a971d2ae53.png  
  inflating: test_solt/images/4ad823e2d3.png  
  inflating: test_solt/images/85a9687a4f.png  
  inflating: test_solt/images/da5b430ecd.png  
  inflating: test_solt/images/b1a2903d1c.png  
  inflating: test_solt/images/70958cacd4.png  
  inflating: test_solt/image

In [0]:
# セグメンテーションの為にUnetを使う、Unetをクローンする所から開始する

In [0]:
# Unetモデルをクローンする
! git clone https://github.com/zhixuhao/unet

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.92 MiB | 43.39 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [0]:
# クローンしたUnetが持つデータは不要なので削除する
rm -rf ./unet/data/membrane/train/image/*

In [0]:
rm -rf ./unet/data/membrane/test/*

In [0]:
# 解答したimagesデータをUnetで使用する為にコピーする
!cp ./train_solt/images/* ./unet/data/membrane/train/image

In [0]:
!cp ./test_solt/images/* ./unet/data/membrane/test

In [0]:
# os.chdir はpython上でのディレクトリ移動。unet をcloneして来ているのでunet 
# の中に入る必要がある。
import os

os.chdir('/content/unet')
print(os.getcwd())

/content/unet


In [0]:
# 今回はunet のロジックを使用して塩の画像データをセグメンテーションする

In [0]:
# 使わなくて良い。tensorflowのバージョンが古いと言われた時の対応方法として書いてあったが関係なかった。
# !pip install  tf-nightly-2.0-preview

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense

In [0]:
# ファイル名は連番でないと読めないプログラムになっているので、リネイムする。
# 初めに今のファイル名の状態やレコード数を確認する。
path = '/content/unet/data/membrane/test' 
flist = os.listdir(path)
flist[:10]

['e1207841c3.png',
 '43311d0da8.png',
 '7bca7168b6.png',
 '2862c77d0c.png',
 '3c5e3124c7.png',
 '0915403c23.png',
 '280150526a.png',
 'a496d57c85.png',
 'be72bfb0b2.png',
 'd57d30d6a1.png']

In [0]:
len(flist)

18000

In [0]:
pwd

'/content/unet'

In [0]:
cd /content/unet/data/membrane/test

/content/unet/data/membrane/test


In [0]:
#rename の実施をする。
for i , name in enumerate(flist):
    renamed = str(i)+".jpg"
    os.rename(name, renamed)

In [0]:
path = '/content/unet/data/membrane/test'
flist_renamed = os.listdir(path)
sorted(flist_renamed)[:10]

['0.jpg',
 '1.jpg',
 '10.jpg',
 '100.jpg',
 '1000.jpg',
 '10000.jpg',
 '10001.jpg',
 '10002.jpg',
 '10003.jpg',
 '10004.jpg']

In [0]:
cd ../

/content/unet/data/membrane


In [0]:
!mkdir results

In [0]:
cd /content/unet/

/content/unet


In [0]:
# unetのプログラムを修正する。
# data.pyを修正して、アップロードする。
# def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    # for i in range(num_image): この２行を修正する。
    #     img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)
    # for i in range(num_image):
    #     img = io.imread(os.path.join(test_path,"%d.jpg"%i),as_gray = as_gray)

# main.pyも修正。
# saveResult("data/membrane/test",results)→出力フォルダを変更する。
# saveResult("data/membrane/results",results)
 
!python main.py --epoch_num 2 -i 300 --testsize 100

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1
Found 4000 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


2020-03-23 06:25:32.357796: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-03-23 06:25:32.371613: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000160000 Hz
2020-03-23 06:25:32.371793: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x143ca00 initialized for platform Host (this does not guarantee that XLA will be used). Devic

In [0]:
# resultsフォルダに出力内容が保存されるので内容を確認。（グレーの四角が出力された。。）